In [1]:
#import Keras 
from keras import backend as K
from keras.utils import generic_utils

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.layers import Input,Dropout ,Dense, Conv1D, MaxPooling1D, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.layers import Embedding

import tensorflow as tf
#import pands and nympy for data preprocesing 
import pandas as pd
import numpy as np
#Pickle for loading the big model
import pickle

Using TensorFlow backend.


In [2]:
# splittest set
MAX_NB_WORDS = 5000 # consider to 20,000 most occuring words in dataset
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
# load the model from disk
filename = 'x1.sav'
x1 = pickle.load(open(filename, 'rb'))
global graph
graph = tf.get_default_graph()

In [3]:
#Import twitter API
from twython import Twython
from twython import TwythonStreamer
from twitter import *
APP_KEY=""
APP_SECRET=""
OAUTH_TOKEN=""
OAUTH_TOKEN_SECRET=""
#Telegram API
import telegram
import logging
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
#pymongo library for accesing the database
import pymongo
from pymongo import MongoClient, TEXT, DESCENDING
from datetime import datetime, timedelta
from simple_settings import settings
#Bot information
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)
#Input token from Telegram
updater = Updater(token='')
dispatcher = updater.dispatcher
#set up conection to database
client = MongoClient()
client = MongoClient('localhost', 27017)
#connect to database
db = client.yo
#select collection
cfeedback=db.feedback
#Global variable for storing feedback
fb = ""
class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        if 'text' in data:
            global job
            global model
            sequences= tokenizer.texts_to_sequences([data['text']])
            dat = pad_sequences(sequences, maxlen=1000)
            with graph.as_default():
                prediction1 = x1.predict(dat, batch_size=64)[0,1]
                prediction2 = model.predict(dat, batch_size=64)[0,1]
                prediction= (prediction1+prediction2)/2
        if prediction >0.4:
            job=data
            self.disconnect()

    def on_error(self, status_code, data):
        print(status_code)

stream = MyStreamer(APP_KEY, APP_SECRET,
                    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [4]:
#define comands
#Show add on start
def start(bot, update):
    global updater
    global fb
    global job

    stream.statuses.filter(track='Hiring, CareerArc')
    bot.send_message(chat_id=update.message.chat_id, text="Please rate this 1 or 0")
    bot.send_message(chat_id=update.message.chat_id, text=job["text"])    
    fb=job["text"]

#Collect feedback get text from previous post and insert into database with user feedback
def collect(msg,uid):
    global fb
    global model
#    global job
    feedback={"User_id":uid,
         "text":fb,
         "feedback":msg}    
    cfeedback.insert_one(feedback)
    with graph.as_default():
        retrain()
#For  unknown message and collecting feedback
def echo(bot, update):
    if update.message.text=='1':        
        collect(1,update.message.chat_id)
        bot.send_message(chat_id=update.message.chat_id, text="----------------------------------------->")
        start(bot, update)
    elif update.message.text=='0':
        collect(0,update.message.chat_id)
        bot.send_message(chat_id=update.message.chat_id, text="----------------------------------------->")
        start(bot, update)
    else:
        bot.send_message(chat_id=update.message.chat_id, text="Answer with 1 for true and 0 for false")
        bot.send_message(chat_id=update.message.chat_id, text="----------------------------------------->")
        start(bot, update)

#for unknown comand message.chat_id=id of the user get it dynamicaly
def unknown(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="Sorry, I didn't understand that command.")

#Initiate the comands
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)
#unknown message
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)
#unknown comand
unknown_handler = MessageHandler(Filters.command, unknown)
dispatcher.add_handler(unknown_handler)

In [ ]:
#Start the bot
updater.start_polling()

In [7]:
#Stop the bot
updater.stop()

In [ ]:
#Function for second model
def retrain():
    #We get our collection from the database and save it in a pandas dataframe
    df = pd.DataFrame(list(cfeedback.find()))
    labels = df['feedback']
    texts = df['text']

    MAX_NB_WORDS = 5000 # consider to 20,000 most occuring words in dataset
    MAX_SEQUENCE_LENGTH = 1000 # truncate sequences to a maximum length of 1000 words
    VALIDATION_SPLIT = 0.2

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    labels = to_categorical(labels)

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    word_index = tokenizer.word_index
    # split the data into a training set and a validation set
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices]

    nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

    x_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    x_val = data[-nb_validation_samples:]
    y_val = labels[-nb_validation_samples:]

    embeddings_index = {}

    embedding_layer = Embedding(len(word_index) + 1, 100, input_length=MAX_SEQUENCE_LENGTH)
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    # 3 hidden layers with 128 neurons each
    x = Conv1D(64, 5, activation='relu')(embedded_sequences)
    x = Dropout(0.1)(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(64, 5, activation='relu')(x)
    #x = Dropout(0.3)(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(64, 5, activation='relu')(x)
    #x = Dropout(0.2)(x)
    x = MaxPooling1D(35)(x)  # global max pooling
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    preds = Dense(2, activation='softmax')(x) # 2 ... binary
    global model
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics= ['acc'])
    # 10 epochs
    model_hist = model.fit(x_train, y_train, validation_data=(x_val, y_val),
              epochs=10, batch_size=32)
retrain()